In [22]:
import urllib
import ntpath
from pipelineapi import PipelineAPI
import os
import glob


def path_leaf(path):
    if urllib.parse.urlparse(path).scheme != "" or os.path.isabs(path):
        path = os.path.split(path)[-1]
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

In [23]:
from pipelineapi import PipelineAPI
import os

def dataset_import(dataset_name, dataset_file):
        data_pipeline_api = PipelineAPI("http://ec2-54-191-65-138.us-west-2.compute.amazonaws.com/", 'admin@pronto.ai', 'admin123!')
        with open(dataset_file, 'rb') as f:
            data_pipeline_api.upload_json_file(f, file_name=path_leaf(dataset_file), round='inspection', dataset_name=dataset_name, skip_image=False)
            

In [40]:
import subprocess
from subprocess import Popen,PIPE,STDOUT,call
import hashlib
import shutil

def send_to_s3( img_path, s3_bucket = 'kache-scalabel/kache_ai/frames/'):
    
        hex_val = os.path.abspath(img_path).replace('/', '^^')
        hash_object = hashlib.sha1(str.encode(hex_val))
        hex_dig = hash_object.hexdigest()
        s3_path = os.path.join(s3_bucket,str(hex_dig)+os.path.splitext(img_path)[-1])
        new_path = os.path.join(os.path.split(img_path)[0],str(hex_dig)+os.path.splitext(img_path)[-1])
        shutil.copy(img_path, new_path)

        if True:
            #exists = subprocess.check_output("aws s3 ls {}".format(s3_path), shell=True)
            # if not exists:
            s3_bucket = 's3://'+s3_bucket
            sp = subprocess.Popen("aws s3 cp {} {}".format(new_path, s3_bucket), shell=True, stdout=PIPE)
            out_str = sp.communicate()
            print(out_str[0].decode("utf-8"))
        return os.path.join('https://s3-us-west-2.amazonaws.com', s3_path)

In [41]:
def update_cats(data):
    for img_data in data:
        img_data['urls'] = send_to_s3(img_data['url'])
        if img_data.get('labels', None):
            deletions = []
            for idx, label in enumerate(img_data['labels']):
                # If detected - delete
                if 'detected-' in label['category']:
                    label['category'] = label['category'].replace('detected-', '')
                if 'tracked-' in label['category']:
                    deletions.append(label)
                if 'flickering_' in label['category']:
                    label['category'] = label['category'].replace('flickering_', '').replace('tracked-', '')
            
            for label in deletions:
                img_data['labels'].remove(label)
                
    return data

In [42]:
def strip_kache_data(data):
    for img_data in data:
        kache_id = None
        if img_data.get('labels', None):
            for idx, label in enumerate(img_data['labels']):
                    label['kache_label_id'] = None
    return data

In [ ]:
import json
import glob
for dset in glob.glob('/home/dean/Desktop/071519_flickers/*.json'):
    with open(dset, 'r') as fn:
        data = json.load(fn)

    data = update_cats(data)
    # Strip flicker mining database indexes
    data = strip_kache_data(data)
    
    with open(dset, 'w+') as fn:
        json.dump(data, fn)

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904230917_000005AA_MP4-frames/8d280ae88b8eee1c81f1622080342cfadbf80182.jpg to s3://kache-scalabel/kache_ai/frames/8d280ae88b8eee1c81f1622080342cfadbf80182.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904230917_000005AA_MP4-frames/aca5521afa8ce7450b5dd7cc622576119a7d139d.jpg to s3://kache-scalabel/kache_ai/frames/aca5521afa8ce7450b5dd7cc622576119a7d139d.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904230917_000005AA_MP4-frames/cbc4e849ccec6e41207d38f398e6afa8a95c5cd0.jpg to s3://kache-scalabel/kache_ai/frames/cbc4e849ccec6e41207d38f398e6afa8a95c5cd0.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904230917_000005AA_MP4-frames/2676c77587c839015a2f2f690f7623c07f050e5d.jpg to s3://kache-scalabel/kache_ai/frames/2676c77587c839015a2f2f690f7623c07f050e5d.jpg

upload: 

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904230917_000005AA_MP4-frames/bb663788f2708f6b0a43e902da4d9261c7cdd702.jpg to s3://kache-scalabel/kache_ai/frames/bb663788f2708f6b0a43e902da4d9261c7cdd702.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904230917_000005AA_MP4-frames/b6d87a2e154f2513ab1c3773da6cfe58ecb34934.jpg to s3://kache-scalabel/kache_ai/frames/b6d87a2e154f2513ab1c3773da6cfe58ecb34934.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904230917_000005AA_MP4-frames/537924e3e18c4d5d955616c9336cbc66e328cd7e.jpg to s3://kache-scalabel/kache_ai/frames/537924e3e18c4d5d955616c9336cbc66e328cd7e.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904230917_000005AA_MP4-frames/229b376dc154fda01b6fe6d92c088fabf1a1cdbe.jpg to s3://kache-scalabel/kache_ai/frames/229b376dc154fda01b6fe6d92c088fabf1a1cdbe.jpg

upload: 

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904230917_000005AA_MP4-frames/0f4d424a5f43f4e343b0b462c55ff02d64e407ca.jpg to s3://kache-scalabel/kache_ai/frames/0f4d424a5f43f4e343b0b462c55ff02d64e407ca.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904230917_000005AA_MP4-frames/9fc64e4b7934855353b5b7adfd5f025438523ba8.jpg to s3://kache-scalabel/kache_ai/frames/9fc64e4b7934855353b5b7adfd5f025438523ba8.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904230917_000005AA_MP4-frames/08ad2eb9706b7b96686fa6cbf781e5804f4e1cbd.jpg to s3://kache-scalabel/kache_ai/frames/08ad2eb9706b7b96686fa6cbf781e5804f4e1cbd.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904230917_000005AA_MP4-frames/a436f6d3de6369c72f289037670a6f117f37f9d6.jpg to s3://kache-scalabel/kache_ai/frames/a436f6d3de6369c72f289037670a6f117f37f9d6.jpg

upload: 

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904230917_000005AA_MP4-frames/18f587b0321ac41a4ba7c171c0eb210020417218.jpg to s3://kache-scalabel/kache_ai/frames/18f587b0321ac41a4ba7c171c0eb210020417218.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904230917_000005AA_MP4-frames/96ff27dfacdc644975b7eeacb50a7a8130edfef0.jpg to s3://kache-scalabel/kache_ai/frames/96ff27dfacdc644975b7eeacb50a7a8130edfef0.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904230917_000005AA_MP4-frames/6c1e5a76efeb4f2947e75b32501ac45918612904.jpg to s3://kache-scalabel/kache_ai/frames/6c1e5a76efeb4f2947e75b32501ac45918612904.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904230917_000005AA_MP4-frames/34c315ffa8a2f429959c476471d69325e98c325d.jpg to s3://kache-scalabel/kache_ai/frames/34c315ffa8a2f429959c476471d69325e98c325d.jpg

upload: 

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904230917_000005AA_MP4-frames/91eda8e98c45869c84d73d172fc9b99f96b02f7c.jpg to s3://kache-scalabel/kache_ai/frames/91eda8e98c45869c84d73d172fc9b99f96b02f7c.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904230917_000005AA_MP4-frames/ed388eda94324eb474130751e1d6df8d712f1ea1.jpg to s3://kache-scalabel/kache_ai/frames/ed388eda94324eb474130751e1d6df8d712f1ea1.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904230917_000005AA_MP4-frames/42a90f4664f838b0e602fc2c2d6bdf1b9063c863.jpg to s3://kache-scalabel/kache_ai/frames/42a90f4664f838b0e602fc2c2d6bdf1b9063c863.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904230917_000005AA_MP4-frames/cb9da944e038129417507dfa4abe5a4b4bb41dc5.jpg to s3://kache-scalabel/kache_ai/frames/cb9da944e038129417507dfa4abe5a4b4bb41dc5.jpg

upload: 

...

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904260737_000045AA_MP4-frames/3a8d3e6cb0c036e9dd18fac2b3f814054e679195.jpg to s3://kache-scalabel/kache_ai/frames/3a8d3e6cb0c036e9dd18fac2b3f814054e679195.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904260737_000045AA_MP4-frames/26eca0b5181176db1ae1111bd324cc70fe2ecf23.jpg to s3://kache-scalabel/kache_ai/frames/26eca0b5181176db1ae1111bd324cc70fe2ecf23.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904260737_000045AA_MP4-frames/40a43d17365e98e1b897a965ba2842e6c520cda3.jpg to s3://kache-scalabel/kache_ai/frames/40a43d17365e98e1b897a965ba2842e6c520cda3.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904260737_000045AA_MP4-frames/39ea6be2e7be2709764330aab614d6f357ffd157.jpg to s3://kache-scalabel/kache_ai/frames/39ea6be2e7be2709764330aab614d6f357ffd157.jpg

upload: 

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904260737_000045AA_MP4-frames/fcaaf5c27a21998373c3b70cace9f9d7cd33baa9.jpg to s3://kache-scalabel/kache_ai/frames/fcaaf5c27a21998373c3b70cace9f9d7cd33baa9.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904260737_000045AA_MP4-frames/92eef2a16bc07909cb16b01c06ca980f26dc4d13.jpg to s3://kache-scalabel/kache_ai/frames/92eef2a16bc07909cb16b01c06ca980f26dc4d13.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904260737_000045AA_MP4-frames/162fe84f7b5b4fc516c4485a039af61a591006d1.jpg to s3://kache-scalabel/kache_ai/frames/162fe84f7b5b4fc516c4485a039af61a591006d1.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904260737_000045AA_MP4-frames/1e70a1fbaf7ed6cdcea3fad018191346b6fdd269.jpg to s3://kache-scalabel/kache_ai/frames/1e70a1fbaf7ed6cdcea3fad018191346b6fdd269.jpg

upload: 

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904260737_000045AA_MP4-frames/ae729cee7cd54992bf05c0cad7a8c8954045092e.jpg to s3://kache-scalabel/kache_ai/frames/ae729cee7cd54992bf05c0cad7a8c8954045092e.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904260737_000045AA_MP4-frames/9c343ce87733bf84079fa136bade4240fd74f6ef.jpg to s3://kache-scalabel/kache_ai/frames/9c343ce87733bf84079fa136bade4240fd74f6ef.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904260737_000045AA_MP4-frames/bdb99a1c14ea12d76f232af6d5d877ad42a8d1e1.jpg to s3://kache-scalabel/kache_ai/frames/bdb99a1c14ea12d76f232af6d5d877ad42a8d1e1.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904260737_000045AA_MP4-frames/479f0062a5ff6997f9bcf2ceaa10eb4ce5a7dd3b.jpg to s3://kache-scalabel/kache_ai/frames/479f0062a5ff6997f9bcf2ceaa10eb4ce5a7dd3b.jpg

upload: 

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904260737_000045AA_MP4-frames/34730cb992e5480ed2ff8de66bd0b7897522fe4e.jpg to s3://kache-scalabel/kache_ai/frames/34730cb992e5480ed2ff8de66bd0b7897522fe4e.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904260737_000045AA_MP4-frames/3e99c085f7fbdfa5af4737cd64316ec799248d18.jpg to s3://kache-scalabel/kache_ai/frames/3e99c085f7fbdfa5af4737cd64316ec799248d18.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904260737_000045AA_MP4-frames/b5b431189165bff431ddd19c989a62173747169d.jpg to s3://kache-scalabel/kache_ai/frames/b5b431189165bff431ddd19c989a62173747169d.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904260737_000045AA_MP4-frames/20cc115b2251c892f69a610a17d13159c114f510.jpg to s3://kache-scalabel/kache_ai/frames/20cc115b2251c892f69a610a17d13159c114f510.jpg

upload: 

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904260737_000045AA_MP4-frames/e56bd8ba9b00c821c3cb0957fd706e9f1a40018a.jpg to s3://kache-scalabel/kache_ai/frames/e56bd8ba9b00c821c3cb0957fd706e9f1a40018a.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904260737_000045AA_MP4-frames/c269baba192654e47a270a702b2932d8ccc5f3f6.jpg to s3://kache-scalabel/kache_ai/frames/c269baba192654e47a270a702b2932d8ccc5f3f6.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904260737_000045AA_MP4-frames/59d549c7396ec591d03b38484245228df8324d13.jpg to s3://kache-scalabel/kache_ai/frames/59d549c7396ec591d03b38484245228df8324d13.jpg

upload: ../../../media/dean/datastore/nebula/pipelines/kache/frames/external_sources/201904260737_000045AA_MP4-frames/75846535f416f914d18ddd40b2f1c54090e9e330.jpg to s3://kache-scalabel/kache_ai/frames/75846535f416f914d18ddd40b2f1c54090e9e330.jpg

upload: 

In [ ]:
for idx, file in enumerate(glob.glob('/home/dean/Desktop/071519_flickers/*.json')):
    
   

    dataset_name = f'demo_set_{idx}'
    print(f'Importing file to db: {file} and dataset {dataset_name}')

    dataset_import('demo_set', file)

In [ ]:
model_dir = '/media/kuser/rack2remote/nebula/pipelines/best_models'

In [ ]:
from utils import format_data as datasets


for file in glob.glob('/media/kuser/Data/06182019_trn/*flickers*')[4:]:
    
    print(f'Importing file to db: {file}')

    dataset_name = 'pronto_set'
    
    

In [ ]:
import urllib
import ntpath
import json
import os

def get_dataset():
    data_pipeline_api = PipelineAPI("http://ec2-54-149-236-75.us-west-2.compute.amazonaws.com/", 'admin@pronto.ai', 'admin123!')

    batch_size = 0
    batch = 0
    batch_anns = []


    dataset_names = ['062019', '032019']
    kache_anns_dir =  '/home/dean/Desktop'

    databunch = data_pipeline_api.data(dataset_names=dataset_names, video_source_names=[], categories_names=[],
                    label_statuses=[10], page_size=None, frame_id=None) # returns generator

    for anns in databunch:
#           anns = update_traffic_light_cats([anns])
        batch_anns.append(anns)
        batch_size += len(anns)
        if batch_size % 5000 == 0:
            with open(os.path.join(kache_anns_dir,f'{dataset_names}_{batch}--inspected.json'), 'w') as fn:
                json.dump(batch_anns, fn)
            batch_anns = []
            batch += 1

    # Save any remaining frames
    if len(batch_anns) != 0:
        with open(os.path.join(kache_anns_dir,f'{dataset_names}_{batch}--inspected.json'), 'w') as fn:
            json.dump(batch_anns, fn)


In [ ]:
import urllib
import ntpath
import json
import os

def get_dataset():
    data_pipeline_api = PipelineAPI("http://ec2-54-149-236-75.us-west-2.compute.amazonaws.com/", 'admin@pronto.ai', 'admin123!')

    batch_size = 0
    batch = 0
    batch_anns = []


    dataset_names = ['062019', '032019']
    kache_anns_dir =  '/home/dean/Desktop'

    databunch = data_pipeline_api.data(dataset_names=dataset_names, video_source_names=[], categories_names=['motor','motorbike','motorcycle'],
                    label_statuses=[], page_size=None, frame_id=None) # returns generator

    for anns in databunch:
#           anns = update_traffic_light_cats([anns])
        batch_anns.append(anns)
        batch_size += len(anns)
        if batch_size % 25000 == 0:
            with open(os.path.join(kache_anns_dir,f'{"^".join(dataset_names)}_{batch}--motorcycles.json'), 'w') as fn:
                json.dump(batch_anns, fn)
            batch_anns = []
            batch += 1

    # Save any remaining frames
    if len(batch_anns) != 0:
        with open(os.path.join(kache_anns_dir,f'{"^".join(dataset_names)}_{batch}--motorcycles.json'), 'w') as fn:
            json.dump(batch_anns, fn)


In [ ]:
get_dataset()

In [ ]:
def get_dataset():
    data_pipeline_api = PipelineAPI("http://ec2-54-191-65-138.us-west-2.compute.amazonaws.com/", 'admin@pronto.ai', 'admin123!')

    batch_size = 0
    batch = 0
    batch_anns = []


    dataset_names = 'china_set'
    kache_anns_dir =  '/home/dean/Desktop'

    databunch = data_pipeline_api.data(dataset_names=dataset_names, video_source_names=[], categories_names=[],
                    label_statuses=[20], page_size=None, frame_id=None) # returns generator

    for anns in databunch:
#             anns = update_traffic_light_cats([anns])
        batch_anns.append(anns)
        batch_size += len(anns)
        if batch_size % 5000 == 0:
            with open(os.path.join(kache_anns_dir,f'{dataset_names}_{batch}--rejected.json'), 'w') as fn:
                json.dump(batch_anns, fn)
            batch_anns = []
            batch += 1

    # Save any remaining frames
    if len(batch_anns) != 0:
        with open(os.path.join(kache_anns_dir,f'{dataset_names}_{batch}--rejected.json'), 'w') as fn:
            json.dump(batch_anns, fn)


In [ ]:
get_dataset()

In [9]:
from collections import defaultdict
import numpy as np

attr2idx = {}
idx2attr = {}

In [10]:
attr2idx['video_source'] = 0
attr2idx['cam_num'] = 1
attr2idx['latitude'] = 2
attr2idx['longitude'] = 3
attr2idx['hour_of_day'] = 4

# Create vector that stores a representation of the object distribution from the log, sorted by class, then type, then box size (in descending order)
classes = ["bike", "bus", "car", "construct-equipment", "construct-pole", "construct-post", "construct-sign", "motorcycle", "person", "rider", "traffic light", "traffic light-amber", "traffic light-green", "traffic light-red", "traffic sign", "traffic sign-slow_sign", "traffic sign-speed_sign", "traffic sign-stop_sign", "trailer", "train", "truck",]
#
idx = 5
for cls in classes:
    for tracking_type in ['detected', 'tracked', 'flickering_tracked']:
        print(f'total_{tracking_type}-{cls}',':',idx)
        attr2idx[f'total_{tracking_type}-{cls}'] = idx
        idx += 1

for cls in classes:
    for tracking_type in ['detected', 'tracked', 'flickering_tracked']:
        # Load the first 10 objects in each category
        for i in np.arange(1, 11):
            
            #x1 
            print(f'{tracking_type}-{cls}_{i}_x1',':',idx)
            attr2idx[f'{tracking_type}-{cls}_{i}_x1'] = idx
            idx += 1
            
            #y1 
            print(f'{tracking_type}-{cls}_{i}_y1',':',idx)
            attr2idx[f'{tracking_type}-{cls}_{i}_y1'] = idx
            idx += 1
            
            #x2 
            print(f'{tracking_type}-{cls}_{i}_x2',':',idx)
            attr2idx[f'{tracking_type}-{cls}_{i}_x2'] = idx
            idx += 1
            
            #y2 
            print(f'{tracking_type}-{cls}_{i}_y2',':',idx)
            attr2idx[f'{tracking_type}-{cls}_{i}_y2'] = idx
            idx += 1

idx2attr = {v_i: v for v, v_i in attr2idx.items()}          
    

total_detected-bike : 5
total_tracked-bike : 6
total_flickering_tracked-bike : 7
total_detected-bus : 8
total_tracked-bus : 9
total_flickering_tracked-bus : 10
total_detected-car : 11
total_tracked-car : 12
total_flickering_tracked-car : 13
total_detected-construct-equipment : 14
total_tracked-construct-equipment : 15
total_flickering_tracked-construct-equipment : 16
total_detected-construct-pole : 17
total_tracked-construct-pole : 18
total_flickering_tracked-construct-pole : 19
total_detected-construct-post : 20
total_tracked-construct-post : 21
total_flickering_tracked-construct-post : 22
total_detected-construct-sign : 23
total_tracked-construct-sign : 24
total_flickering_tracked-construct-sign : 25
total_detected-motorcycle : 26
total_tracked-motorcycle : 27
total_flickering_tracked-motorcycle : 28
total_detected-person : 29
total_tracked-person : 30
total_flickering_tracked-person : 31
total_detected-rider : 32
total_tracked-rider : 33
total_flickering_tracked-rider : 34
total_det

flickering_tracked-person_10_y2 : 1147
detected-rider_1_x1 : 1148
detected-rider_1_y1 : 1149
detected-rider_1_x2 : 1150
detected-rider_1_y2 : 1151
detected-rider_2_x1 : 1152
detected-rider_2_y1 : 1153
detected-rider_2_x2 : 1154
detected-rider_2_y2 : 1155
detected-rider_3_x1 : 1156
detected-rider_3_y1 : 1157
detected-rider_3_x2 : 1158
detected-rider_3_y2 : 1159
detected-rider_4_x1 : 1160
detected-rider_4_y1 : 1161
detected-rider_4_x2 : 1162
detected-rider_4_y2 : 1163
detected-rider_5_x1 : 1164
detected-rider_5_y1 : 1165
detected-rider_5_x2 : 1166
detected-rider_5_y2 : 1167
detected-rider_6_x1 : 1168
detected-rider_6_y1 : 1169
detected-rider_6_x2 : 1170
detected-rider_6_y2 : 1171
detected-rider_7_x1 : 1172
detected-rider_7_y1 : 1173
detected-rider_7_x2 : 1174
detected-rider_7_y2 : 1175
detected-rider_8_x1 : 1176
detected-rider_8_y1 : 1177
detected-rider_8_x2 : 1178
detected-rider_8_y2 : 1179
detected-rider_9_x1 : 1180
detected-rider_9_y1 : 1181
detected-rider_9_x2 : 1182
detected-rider_9

flickering_tracked-trailer_2_x2 : 2314
flickering_tracked-trailer_2_y2 : 2315
flickering_tracked-trailer_3_x1 : 2316
flickering_tracked-trailer_3_y1 : 2317
flickering_tracked-trailer_3_x2 : 2318
flickering_tracked-trailer_3_y2 : 2319
flickering_tracked-trailer_4_x1 : 2320
flickering_tracked-trailer_4_y1 : 2321
flickering_tracked-trailer_4_x2 : 2322
flickering_tracked-trailer_4_y2 : 2323
flickering_tracked-trailer_5_x1 : 2324
flickering_tracked-trailer_5_y1 : 2325
flickering_tracked-trailer_5_x2 : 2326
flickering_tracked-trailer_5_y2 : 2327
flickering_tracked-trailer_6_x1 : 2328
flickering_tracked-trailer_6_y1 : 2329
flickering_tracked-trailer_6_x2 : 2330
flickering_tracked-trailer_6_y2 : 2331
flickering_tracked-trailer_7_x1 : 2332
flickering_tracked-trailer_7_y1 : 2333
flickering_tracked-trailer_7_x2 : 2334
flickering_tracked-trailer_7_y2 : 2335
flickering_tracked-trailer_8_x1 : 2336
flickering_tracked-trailer_8_y1 : 2337
flickering_tracked-trailer_8_x2 : 2338
flickering_tracked-traile

In [11]:
print(idx2attr)

{0: 'video_source', 1: 'cam_num', 2: 'latitude', 3: 'longitude', 4: 'hour_of_day', 5: 'total_detected-bike', 6: 'total_tracked-bike', 7: 'total_flickering_tracked-bike', 8: 'total_detected-bus', 9: 'total_tracked-bus', 10: 'total_flickering_tracked-bus', 11: 'total_detected-car', 12: 'total_tracked-car', 13: 'total_flickering_tracked-car', 14: 'total_detected-construct-equipment', 15: 'total_tracked-construct-equipment', 16: 'total_flickering_tracked-construct-equipment', 17: 'total_detected-construct-pole', 18: 'total_tracked-construct-pole', 19: 'total_flickering_tracked-construct-pole', 20: 'total_detected-construct-post', 21: 'total_tracked-construct-post', 22: 'total_flickering_tracked-construct-post', 23: 'total_detected-construct-sign', 24: 'total_tracked-construct-sign', 25: 'total_flickering_tracked-construct-sign', 26: 'total_detected-motorcycle', 27: 'total_tracked-motorcycle', 28: 'total_flickering_tracked-motorcycle', 29: 'total_detected-person', 30: 'total_tracked-person'

In [12]:
a = dict.fromkeys(attr2idx.keys(),0)
print(a)

{'video_source': 0, 'cam_num': 0, 'latitude': 0, 'longitude': 0, 'hour_of_day': 0, 'total_detected-bike': 0, 'total_tracked-bike': 0, 'total_flickering_tracked-bike': 0, 'total_detected-bus': 0, 'total_tracked-bus': 0, 'total_flickering_tracked-bus': 0, 'total_detected-car': 0, 'total_tracked-car': 0, 'total_flickering_tracked-car': 0, 'total_detected-construct-equipment': 0, 'total_tracked-construct-equipment': 0, 'total_flickering_tracked-construct-equipment': 0, 'total_detected-construct-pole': 0, 'total_tracked-construct-pole': 0, 'total_flickering_tracked-construct-pole': 0, 'total_detected-construct-post': 0, 'total_tracked-construct-post': 0, 'total_flickering_tracked-construct-post': 0, 'total_detected-construct-sign': 0, 'total_tracked-construct-sign': 0, 'total_flickering_tracked-construct-sign': 0, 'total_detected-motorcycle': 0, 'total_tracked-motorcycle': 0, 'total_flickering_tracked-motorcycle': 0, 'total_detected-person': 0, 'total_tracked-person': 0, 'total_flickering_t

In [13]:
!sleep 10400

^C


In [14]:
import os
import glob
import datetime
import ast
import json

run_eval = True
if run_eval:
    with open('/home/kuser/Desktop/master_log_features_070219.json', 'w+') as fn:
        json_str = json.dumps([])
        fn.write(json_str)
    
    with open('/home/kuser/Desktop/master_log_features_070219.json', 'r') as fn:
        master_feats = json.load(fn)
            
    for flickers in glob.glob(os.path.join('/media/kuser/Data/','070219_trn','*.json')):
        print(flickers)
        # Collect attributes
        with open(flickers, 'r') as fn:
            flicker_data = json.loads(fn.read())

        if flicker_data and flicker_data[0].get('videoName', None) and 'unknown' not in flicker_data[0]['videoName'].lower():
            feat_vec = dict.fromkeys(attr2idx.keys(), '0.0')
            if 'cam' in flicker_data[0]['videoName']:
                cam_num = int(float(flicker_data[0]['videoName'].split('cam')[-1]))
                feat_vec["cam_num"] = str(cam_num)


            



            # Generate vector
            feat_vec["video_source"] = flicker_data[0]['videoName']


            if flicker_data[0].get('latitude', None):    
                feat_vec["latitude"] = str(flicker_data[0]['latitude'])

            if flicker_data[0].get('longitude', None):    
                feat_vec["longitude"] = str(flicker_data[0]['longitude'])

            if flicker_data[0].get('timestamp', None):
                timestamp = int(flicker_data[0]['timestamp'])
                timestamp_s = timestamp/1e9

                hour = datetime.datetime.fromtimestamp(timestamp_s).strftime('%H')            
                feat_vec["hour_of_day"] = hour



            for flicker_frame in flicker_data:
                flicker_frame_detections = []
                for label in flicker_frame.get('labels'):
                    feat_vec[f"total_{label['category']}"] = str( int( float(feat_vec[f"total_{label['category']}"])) +1 )

                    label_area =  np.abs((label['box2d']['x2'])-(label['box2d']['x1']) * (label['box2d']['y2'])-(label['box2d']['y1']))                
                    flicker_frame_detections.append([f"{label['category']}", label_area, [label["box2d"]["x1"], label["box2d"]["y1"],label["box2d"]["x2"], label["box2d"]["y2"]]] )

                # Fill in detections
                for cls in classes:
                    for tracking_type in ['detected', 'tracked', 'flickering_tracked']:

                        class_dets = [det for det in flicker_frame_detections if f'{tracking_type}-{cls}' in det[0]]

                        # Sort class_dets by area size
                        sorted_class_dets = (sorted(class_dets, key = lambda x: x[1]))

                        # Only process 10 object maximum
                        while len(sorted_class_dets) <= 10:
                            sorted_class_dets.append([f'{tracking_type}-{cls}', 0.0, [0.0, 0.0, 0.0, 0.0]])

                        if len(sorted_class_dets) > 10:
                             sorted_class_dets = sorted_class_dets[:10]

                        # Load the first 10 objects in each category
                        for i, det in enumerate(sorted_class_dets, 1):
                            #x1 
                            feat_vec[f"{tracking_type}-{cls}_{i}_x1"] = str(det[2][0])

                            #y1 
                            feat_vec[f"{tracking_type}-{cls}_{i}_y1"] = str(det[2][1])

                            #x2 
                            feat_vec[f"{tracking_type}-{cls}_{i}_x2"] = str(det[2][2])

                            #y2 
                            feat_vec[f"{tracking_type}-{cls}_{i}_y2"] = str(det[2][3])



            flicker_data[0]['feature_vector'] = feat_vec
            master_feats.append(feat_vec)        







            # Save vector in json file
            with open(flickers, 'w+') as fn:
                json.dump(flicker_data, fn)
                                                     
        else:
            print(f'Skipping {flickers}')
                                                     
                                                     
                                                     
                                                     
    # Save vector in json file
    with open('/home/kuser/Desktop/master_log_features_070219.json', 'w+') as fn:
        json_str = json.dumps( master_feats )
        fn.write(json_str)



        

/media/kuser/Data/070219_trn/0-frames_032019_flickers.json
/media/kuser/Data/070219_trn/032019_0--flickers.json
/media/kuser/Data/070219_trn/1-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1001-frames_032019_flickers.json
Skipping /media/kuser/Data/070219_trn/1001-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1003-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1004-frames_032019_flickers.json
Skipping /media/kuser/Data/070219_trn/1004-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1005-frames_032019_flickers.json
Skipping /media/kuser/Data/070219_trn/1005-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1007-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1008-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1009-frames_032019_flickers.json
/media/kuser/Data/070219_trn/101-frames_032019_flickers.json
Skipping /media/kuser/Data/070219_trn/101-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1010-frames_032019_fli

/media/kuser/Data/070219_trn/bagfile_18-12-22_1225_2_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-12-22_1225_3_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-12-22_2245_0_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-12-22_2245_1_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-12-22_2245_2_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-12-22_2245_3_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-12-22_2245_4_032019_flickers.json
Skipping /media/kuser/Data/070219_trn/bagfile_18-12-22_2245_4_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-12-22_2245_6_032019_flickers.json
Skipping /media/kuser/Data/070219_trn/bagfile_18-12-22_2245_6_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-12-23_1219_0_032019_flickers.json
Skipping /media/kuser/Data/070219_trn/bagfile_18-12-23_1219_0_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-12-23_1219_1_032019_flickers.json
/media/kuse

/media/kuser/Data/070219_trn/bagfile_18-10-07_0956_1_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-10-07_0956_20_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-11-27_1123_5_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-11-27_1123_7_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-11-27_1123_8_032019_flickers.json
Skipping /media/kuser/Data/070219_trn/bagfile_18-11-27_1123_8_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-11-27_1303_10_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-11-27_1303_11_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-11-27_1303_13_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-11-27_1303_14_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-11-27_1303_19_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-11-27_1303_2_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-11-27_1303_3_032019_flickers.json
/media/kuser/Data/07021

/media/kuser/Data/070219_trn/bagfile_18-10-01_1006_59_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-10-26_1120_29_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-10-26_1120_30_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-10-26_1120_31_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-10-26_1120_35_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-10-26_1120_36_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-10-26_1120_37_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-10-26_1120_3_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-10-26_1120_43_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-10-26_1120_44_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-10-26_1120_45_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-10-26_1120_47_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-10-26_1120_48_032019_flickers.json
Skipping /media/kuser/Data

/media/kuser/Data/070219_trn/bagfile_18-09-18_0905_141_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-09-18_0905_143_032019_flickers.json
Skipping /media/kuser/Data/070219_trn/bagfile_18-09-18_0905_143_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-09-18_0905_146_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-09-18_0905_147_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-09-18_0905_150_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-09-18_0905_151_032019_flickers.json
Skipping /media/kuser/Data/070219_trn/bagfile_18-09-18_0905_151_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-09-18_0905_155_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-09-18_0905_156_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-09-18_0905_157_032019_flickers.json
/media/kuser/Data/070219_trn/bagfile_18-09-18_0905_158_032019_flickers.json
Skipping /media/kuser/Data/070219_trn/bagfile_18-09-18_0905_158_032019

...

/media/kuser/Data/070219_trn/469-frames_032019_flickers.json
/media/kuser/Data/070219_trn/529-frames_032019_flickers.json
/media/kuser/Data/070219_trn/579-frames_032019_flickers.json
Skipping /media/kuser/Data/070219_trn/579-frames_032019_flickers.json
/media/kuser/Data/070219_trn/602-frames_032019_flickers.json
/media/kuser/Data/070219_trn/628-frames_032019_flickers.json
/media/kuser/Data/070219_trn/66-frames_032019_flickers.json
/media/kuser/Data/070219_trn/688-frames_032019_flickers.json
/media/kuser/Data/070219_trn/725-frames_032019_flickers.json
/media/kuser/Data/070219_trn/756-frames_032019_flickers.json
/media/kuser/Data/070219_trn/775-frames_032019_flickers.json
/media/kuser/Data/070219_trn/804-frames_032019_flickers.json
/media/kuser/Data/070219_trn/850-frames_032019_flickers.json
/media/kuser/Data/070219_trn/894-frames_032019_flickers.json
/media/kuser/Data/070219_trn/945-frames_032019_flickers.json
/media/kuser/Data/070219_trn/975-frames_032019_flickers.json
/media/kuser/Dat

/media/kuser/Data/070219_trn/586-frames_032019_flickers.json
/media/kuser/Data/070219_trn/587-frames_032019_flickers.json
/media/kuser/Data/070219_trn/588-frames_032019_flickers.json
/media/kuser/Data/070219_trn/589-frames_032019_flickers.json
/media/kuser/Data/070219_trn/590-frames_032019_flickers.json
/media/kuser/Data/070219_trn/591-frames_032019_flickers.json
/media/kuser/Data/070219_trn/593-frames_032019_flickers.json
/media/kuser/Data/070219_trn/594-frames_032019_flickers.json
Skipping /media/kuser/Data/070219_trn/594-frames_032019_flickers.json
/media/kuser/Data/070219_trn/596-frames_032019_flickers.json
/media/kuser/Data/070219_trn/597-frames_032019_flickers.json
/media/kuser/Data/070219_trn/598-frames_032019_flickers.json
/media/kuser/Data/070219_trn/6-frames_032019_flickers.json
/media/kuser/Data/070219_trn/600-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1951-frames_032019_flickers.json
Skipping /media/kuser/Data/070219_trn/1951-frames_032019_flickers.json
/media

/media/kuser/Data/070219_trn/108-frames_032019_flickers.json
Skipping /media/kuser/Data/070219_trn/108-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1082-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1087-frames_032019_flickers.json
Skipping /media/kuser/Data/070219_trn/1087-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1088-frames_032019_flickers.json
Skipping /media/kuser/Data/070219_trn/1088-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1864-frames_032019_flickers.json
Skipping /media/kuser/Data/070219_trn/1864-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1866-frames_032019_flickers.json
Skipping /media/kuser/Data/070219_trn/1866-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1869-frames_032019_flickers.json
/media/kuser/Data/070219_trn/187-frames_032019_flickers.json
Skipping /media/kuser/Data/070219_trn/187-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1873-frames_032019_flickers.json
Skipping /media/kuse

/media/kuser/Data/070219_trn/648-frames_032019_flickers.json
/media/kuser/Data/070219_trn/649-frames_032019_flickers.json
/media/kuser/Data/070219_trn/65-frames_032019_flickers.json
Skipping /media/kuser/Data/070219_trn/65-frames_032019_flickers.json
/media/kuser/Data/070219_trn/651-frames_032019_flickers.json
/media/kuser/Data/070219_trn/652-frames_032019_flickers.json
/media/kuser/Data/070219_trn/656-frames_032019_flickers.json
/media/kuser/Data/070219_trn/658-frames_032019_flickers.json
/media/kuser/Data/070219_trn/659-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1756-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1759-frames_032019_flickers.json
/media/kuser/Data/070219_trn/176-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1762-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1764-frames_032019_flickers.json
/media/kuser/Data/070219_trn/177-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1770-frames_032019_flickers.json
/media/kuser

/media/kuser/Data/070219_trn/1293-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1294-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1296-frames_032019_flickers.json
/media/kuser/Data/070219_trn/13-frames_032019_flickers.json
/media/kuser/Data/070219_trn/130-frames_032019_flickers.json
Skipping /media/kuser/Data/070219_trn/130-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1308-frames_032019_flickers.json
/media/kuser/Data/070219_trn/1309-frames_032019_flickers.json
/media/kuser/Data/070219_trn/330-frames_032019_flickers.json
/media/kuser/Data/070219_trn/332-frames_032019_flickers.json
/media/kuser/Data/070219_trn/333-frames_032019_flickers.json
/media/kuser/Data/070219_trn/334-frames_032019_flickers.json
/media/kuser/Data/070219_trn/337-frames_032019_flickers.json
/media/kuser/Data/070219_trn/339-frames_032019_flickers.json
/media/kuser/Data/070219_trn/342-frames_032019_flickers.json
/media/kuser/Data/070219_trn/343-frames_032019_flickers.json
/media/kuse

In [15]:
with open('/home/kuser/Desktop/master_log_features_070219.json', 'r') as fn:
    master_feats =  json.load(fn)
    log_source_names = [list(v.values())[0] for idx,v in enumerate(master_feats)] 
    X = np.array([list(v.values())[1:] for idx,v in enumerate(master_feats)], dtype=np.float32)
    
    

In [16]:
print(X) ; print(X.shape); print(log_source_names)

[[   0.          0.          0.       ...    0.          0.
     0.      ]
 [   0.         37.651455 -121.879135 ...    0.          0.
     0.      ]
 [   0.          0.          0.       ...    0.          0.
     0.      ]
 ...
 [   0.          0.          0.       ...    0.          0.
     0.      ]
 [   0.          0.          0.       ...    0.          0.
     0.      ]
 [   0.          0.          0.       ...    0.          0.
     0.      ]]
(7380, 2587)
['0-frames', 'bagfile_19-01-13_2046_2', '1-frames', '1003-frames', '1007-frames', '1008-frames', '1009-frames', '1010-frames', '1020-frames', '1025-frames', '1027-frames', '1028-frames', '1029-frames', '1031-frames', '1032-frames', 'bagfile_18-11-17_1120_0', 'bagfile_18-11-17_1120_2', 'bagfile_18-11-17_1120_7', 'bagfile_18-11-17_1120_8', 'bagfile_18-11-17_1141_2', 'bagfile_18-11-17_1141_3', 'bagfile_18-11-17_1208_2', 'bagfile_18-11-18_0859_0', 'bagfile_18-11-18_0859_12', 'bagfile_18-11-18_0859_15', 'bagfile_18-11-18_0859_16',

In [17]:
from sklearn.cluster import MiniBatchKMeans
from sklearn import preprocessing

mm_scaler = preprocessing.MinMaxScaler()
X_scaled = mm_scaler.fit_transform(X)

kmeans = MiniBatchKMeans(n_clusters=30,
                          random_state=0,
                          batch_size=10,
                          max_iter=100).fit(X_scaled)


In [18]:
print(kmeans.cluster_centers_)
print(len(kmeans.cluster_centers_))

[[0.1875     0.46966127 0.28094268 ... 0.         0.         0.        ]
 [0.16666667 0.5409183  0.20468263 ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.625      0.5372593  0.2636516  ... 0.         0.         0.        ]
 [0.25       0.5325417  0.20585817 ... 0.         0.         0.        ]
 [0.5        0.26966858 0.58824134 ... 0.         0.         0.        ]]
30


In [19]:
groups = kmeans.predict(X_scaled)

In [20]:
clustered_logs = []

for idx, group in enumerate(groups):
    print(f"log: {log_source_names[idx]} | cluster: {group}")
    clustered_logs.append((log_source_names[idx], group))

log: 0-frames | cluster: 14
log: bagfile_19-01-13_2046_2 | cluster: 21
log: 1-frames | cluster: 12
log: 1003-frames | cluster: 12
log: 1007-frames | cluster: 4
log: 1008-frames | cluster: 9
log: 1009-frames | cluster: 26
log: 1010-frames | cluster: 7
log: 1020-frames | cluster: 13
log: 1025-frames | cluster: 25
log: 1027-frames | cluster: 12
log: 1028-frames | cluster: 26
log: 1029-frames | cluster: 23
log: 1031-frames | cluster: 15
log: 1032-frames | cluster: 16
log: bagfile_18-11-17_1120_0 | cluster: 3
log: bagfile_18-11-17_1120_2 | cluster: 22
log: bagfile_18-11-17_1120_7 | cluster: 9
log: bagfile_18-11-17_1120_8 | cluster: 11
log: bagfile_18-11-17_1141_2 | cluster: 25
log: bagfile_18-11-17_1141_3 | cluster: 4
log: bagfile_18-11-17_1208_2 | cluster: 4
log: bagfile_18-11-18_0859_0 | cluster: 26
log: bagfile_18-11-18_0859_12 | cluster: 14
log: bagfile_18-11-18_0859_15 | cluster: 13
log: bagfile_18-11-18_0859_16 | cluster: 1
log: bagfile_18-11-18_0859_18 | cluster: 13
log: bagfile_18-1

log: daisy-2019-04-21--21-47-58--cam2 | cluster: 1
log: daisy-2019-04-21--21-49-58 | cluster: 4
log: chrysippus-2019-05-22--17-07-12--cam1 | cluster: 26
log: chrysippus-2019-05-22--17-17-12--cam1 | cluster: 22
log: chrysippus-2019-05-22--17-26-12--cam1 | cluster: 22
log: alanturing-2019-02-15--17-20-17--cam0 | cluster: 4
log: alanturing-2019-02-15--17-24-17--cam1 | cluster: 0
log: alanturing-2019-02-15--17-49-17--cam1 | cluster: 10
log: alanturing-2019-02-15--18-14-17--cam1 | cluster: 7
log: alanturing-2019-02-05--14-58-43 | cluster: 12
log: alanturing-2019-02-05--14-59-43--cam0 | cluster: 23
log: alanturing-2019-02-05--15-06-43--cam0 | cluster: 5
log: alanturing-2019-02-05--15-22-43--cam0 | cluster: 4
log: alanturing-2019-02-05--15-27-43--cam1 | cluster: 25
log: alanturing-2019-02-05--15-51-43--cam1 | cluster: 4
log: alanturing-2019-02-05--16-01-43--cam1 | cluster: 13
log: alanturing-2019-02-05--16-19-43--cam1 | cluster: 14
log: alanturing-2019-03-01--14-58-28--cam1 | cluster: 26
log:

log: alanturing-2019-05-13--08-27-31--cam0 | cluster: 25
log: alanturing-2019-05-13--08-30-31--cam1 | cluster: 14
log: alanturing-2019-05-13--08-36-58--cam1 | cluster: 1
log: bagfile_18-08-27_1849_0 | cluster: 29
log: alanturing-2019-05-01--16-12-32--cam0 | cluster: 1
log: alanturing-2019-05-01--16-18-32--cam0 | cluster: 1
log: alanturing-2019-05-01--16-40-40--cam1 | cluster: 23
log: alanturing-2019-05-01--16-42-40--cam1 | cluster: 13
log: alanturing-2019-05-01--16-52-40--cam1 | cluster: 1
log: alanturing-2019-05-01--16-53-40--cam0 | cluster: 23
log: alanturing-2019-05-01--16-54-40 | cluster: 13
log: chrysippus-2019-03-22--09-59-29--cam1 | cluster: 6
log: chrysippus-2019-03-22--10-02-29--cam0 | cluster: 20
log: chrysippus-2019-03-22--13-19-08--cam0 | cluster: 0
log: chrysippus-2019-03-22--13-20-08--cam1 | cluster: 10
log: chrysippus-2019-03-22--13-27-08--cam2 | cluster: 26
log: chrysippus-2019-03-22--13-34-08--cam0 | cluster: 4
log: chrysippus-2019-03-22--13-45-08 | cluster: 10
log: da

In [21]:
## Workflow for Quasars

## Supernovae Generates Flickers JSON and pushed message to generate feature vector ##

## Download log from flickering db (get all frames under same video_source_name) ##
## Generate Feature Vector ##
## SDave Feature Vector in Cached Json Flickers File ##



In [22]:
def get_best_log(cluster_matrix, category, attr2idx, log_feat_vectors):
    best_log = None
    best_flickering_cat_element = np.amax(cluster_matrix[:, attr2idx[f'total_flickering_tracked-{category}'] - 1])
    best_tracked_cat_element = np.amax(cluster_matrix[:, attr2idx[f'total_tracked-{category}'] - 1])
    best_detected_cat_element = np.amax(cluster_matrix[:, attr2idx[f'total_detected-{category}'] - 1])
    
    if best_flickering_cat_element > 0:
        print("best_flickering_cat_element", best_flickering_cat_element)
        best_flickering_cat_log_idx = np.where( cluster_matrix[:, attr2idx[f'total_flickering_tracked-{category}'] - 1] == best_flickering_cat_element)[0]

        if type(best_flickering_cat_log_idx) != int:
            if best_flickering_cat_log_idx.any():
                best_flickering_cat_log_idx = best_flickering_cat_log_idx.ravel()[0]
                best_flickering_cat_log_idx = np.asscalar(best_flickering_cat_log_idx)
                
                
                print("best_flickering_cat_log_idx", best_flickering_cat_log_idx)
                best_log_vec = X[best_flickering_cat_log_idx]
                best_log_list = [log for log, vec in log_feat_vectors.items() if np.array_equal(best_log_vec, vec)]
                
                if any(best_log_list):
                    best_log = best_log_list[0]
                
                
#                 best_log = list(log_feat_vectors.keys())[best_flickering_cat_log_idx]
                print("BEST FLICKERING LOG: ", best_log, cluster_matrix[best_flickering_cat_log_idx, attr2idx[f'total_flickering_tracked-{category}'] - 1])
            

    elif best_tracked_cat_element > 0:
        print("best_tracked_cat_element", best_tracked_cat_element)
        best_tracked_cat_log_idx = np.where( cluster_matrix[:, attr2idx[f'total_tracked-{category}'] - 1] == best_tracked_cat_element)[0]

        if type(best_tracked_cat_log_idx) != int:
            if best_tracked_cat_log_idx.any():
                best_tracked_cat_log_idx = best_tracked_cat_log_idx.ravel()[0]
                best_tracked_cat_log_idx = np.asscalar(best_tracked_cat_log_idx)

                print("best_tracked_cat_log_idx", best_tracked_cat_log_idx)
                best_log_vec = X[best_tracked_cat_log_idx]
                best_log_list = [log for log, vec in log_feat_vectors.items() if np.array_equal(best_log_vec, vec)]
                
                if any(best_log_list):
                    best_log = best_log_list[0]
#                 best_log = list(log_feat_vectors.keys())[int(best_tracked_cat_log_idx)]
                print("BEST TRACKED LOG: ", best_log, (cluster_matrix[best_tracked_cat_log_idx, attr2idx[f'total_tracked-{category}'] - 1]))
        
    elif best_detected_cat_element > 0:
        print("best_detected_cat_element", best_detected_cat_element)
        best_detected_cat_log_idx = np.where( cluster_matrix[:, attr2idx[f'total_tracked-{category}'] - 1] == best_detected_cat_element)[0]

        if type(best_detected_cat_log_idx) != int:
            if best_detected_cat_log_idx.any():
                best_detected_cat_log_idx = best_detected_cat_log_idx.ravel()[0]
                best_detected_cat_log_idx = np.asscalar(best_detected_cat_log_idx)
            
                print("best_detected_cat_log_idx", best_detected_cat_log_idx)
                best_log_vec = X[best_detected_cat_log_idx]
                best_log_list = [log for log, vec in log_feat_vectors.items() if np.array_equal(best_log_vec, vec)]
                
                if any(best_log_list):
                    best_log = best_log_list[0]
                
                
#                 best_log = list(log_feat_vectors.keys())[int(best_detected_cat_log_idx)]
                print("BEST DETECTED LOG: ", best_log, (cluster_matrix[best_detected_cat_log_idx, attr2idx[f'total_tracked-{category}'] - 1]))
        
    else:
         print(f"No Best log found for category: {category}.. Skipping")
            
          
    return best_log

In [23]:
from collections import OrderedDict 

log_cluster = defaultdict(list)

for log in clustered_logs:
    log_cluster[log[1]].append(log[0])
    
logs_selected = []
labels  = [ "bike", "bus", "car", "construct-equipment", "construct-pole", "construct-post", "construct-sign", "motorcycle", "person", "rider", "traffic light", "traffic light-amber", "traffic light-green", "traffic light-red", "traffic sign", "traffic sign-slow_sign", "traffic sign-speed_sign", "traffic sign-stop_sign", "trailer", "train", "truck",]
with open('/home/kuser/Desktop/master_log_features_070219.json', 'r') as fn:
    master_feats =  json.load(fn)
        
for cluster, logs in log_cluster.items():
    # Generate table of logs and feature vectors for each cluster
    log_feat_vectors = OrderedDict()
    
    # sort logs by most flickers of category
    # select log with most
    print("\n\nCluster:", cluster)
    
    for log in sorted(logs):
#         with open(glob.glob(os.path.join(f"/home/dean/Desktop/flickers/{log}*"))[0], 'r') as fn:
#             flicker_data = json.loads(fn.read())
        
            
        log_idx = [idx for idx, feat in enumerate(master_feats) if feat['video_source'] == log][0]
        log_feat_vectors[log] = np.array(list(master_feats[log_idx].values())[1:], dtype= np.float32)
        

    daytime_delta = [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17 ,18, 19]
    for cat in labels:
        # Get best daytime log for category
        X = np.array([v for k, v in log_feat_vectors.items() if k not in logs_selected and v[attr2idx['hour_of_day']] in daytime_delta])
        if X.any():
            best_log = get_best_log(X, cat, attr2idx, log_feat_vectors)
            if best_log: 
                logs_selected.append(best_log)
                
        # Get best nighttime log for category
        X = np.array([v for k, v in log_feat_vectors.items() if k not in logs_selected and v[attr2idx['hour_of_day']] not in daytime_delta])
        if X.any():
            best_log = get_best_log(X, cat, attr2idx, log_feat_vectors)
            if best_log: 
                logs_selected.append(best_log)
    
print(set(logs_selected))   
print(len(set(logs_selected)))



Cluster: 14
best_tracked_cat_element 1.0
best_tracked_cat_log_idx 42
BEST TRACKED LOG:  bagfile_18-09-18_0905_91 1.0
best_flickering_cat_element 236.0
best_flickering_cat_log_idx 2
BEST FLICKERING LOG:  2-frames 236.0
best_flickering_cat_element 2912.0
best_flickering_cat_log_idx 80
BEST FLICKERING LOG:  bagfile_18-11-18_1402_0 2912.0
No Best log found for category: construct-equipment.. Skipping
No Best log found for category: construct-pole.. Skipping
No Best log found for category: construct-post.. Skipping
No Best log found for category: construct-sign.. Skipping
best_tracked_cat_element 10.0
best_tracked_cat_log_idx 167
BEST TRACKED LOG:  lily-2019-04-21--12-41-12--cam0 10.0
best_flickering_cat_element 297.0
best_flickering_cat_log_idx 104
BEST FLICKERING LOG:  daisy-2019-04-22--18-50-43 297.0
best_tracked_cat_element 7.0
best_tracked_cat_log_idx 54
BEST TRACKED LOG:  bagfile_18-10-25_1349_4 7.0
best_tracked_cat_element 1045.0
best_tracked_cat_log_idx 183
BEST TRACKED LOG:  lily

BEST FLICKERING LOG:  bagfile_18-10-25_1349_0 41.0
best_tracked_cat_element 64.0
best_tracked_cat_log_idx 1
BEST TRACKED LOG:  chrysippus-2019-01-13--12-59-18 64.0
best_flickering_cat_element 1900.0
best_flickering_cat_log_idx 189
BEST FLICKERING LOG:  bagfile_19-01-10_0744_1 1900.0
No Best log found for category: traffic light-amber.. Skipping
No Best log found for category: traffic light-amber.. Skipping
best_tracked_cat_element 12.0
best_tracked_cat_log_idx 1
BEST TRACKED LOG:  sallyride-2019-02-07--10-42-57--cam1 12.0
best_tracked_cat_element 2019.0
best_tracked_cat_log_idx 309
BEST TRACKED LOG:  sallyride-2019-02-28--23-39-20--cam1 2019.0
best_tracked_cat_element 1.0
best_flickering_cat_element 30.0
best_flickering_cat_log_idx 268
BEST FLICKERING LOG:  daisy-2019-04-22--19-29-44--cam0 30.0
best_tracked_cat_element 888.0
best_flickering_cat_element 1483.0
best_flickering_cat_log_idx 190
BEST FLICKERING LOG:  bagfile_19-01-10_0744_5 1483.0
No Best log found for category: traffic sig

best_flickering_cat_element 1185.0
best_flickering_cat_log_idx 241
BEST FLICKERING LOG:  bagfile_19-01-11_2137_25 1185.0
No Best log found for category: traffic light-amber.. Skipping
No Best log found for category: traffic light-amber.. Skipping
best_tracked_cat_element 1.0
best_flickering_cat_element 10.0
best_flickering_cat_log_idx 253
BEST FLICKERING LOG:  bagfile_19-02-13_0816_1 10.0
best_tracked_cat_element 3.0
best_flickering_cat_element 6218.0
best_flickering_cat_log_idx 134
BEST FLICKERING LOG:  bagfile_18-10-07_0956_4 6218.0
best_tracked_cat_element 444.0
best_flickering_cat_element 3075.0
best_flickering_cat_log_idx 193
BEST FLICKERING LOG:  bagfile_18-11-27_0609_3 3075.0
No Best log found for category: traffic sign-slow_sign.. Skipping
No Best log found for category: traffic sign-slow_sign.. Skipping
No Best log found for category: traffic sign-speed_sign.. Skipping
No Best log found for category: traffic sign-speed_sign.. Skipping
No Best log found for category: traffic si

best_tracked_cat_element 24.0
best_tracked_cat_log_idx 187
BEST TRACKED LOG:  bagfile_18-11-18_1524_4 24.0
best_flickering_cat_element 502.0
best_flickering_cat_log_idx 4
BEST FLICKERING LOG:  13-frames 502.0
best_flickering_cat_element 7314.0
best_flickering_cat_log_idx 164
BEST FLICKERING LOG:  bagfile_18-11-15_1051_35 7314.0
No Best log found for category: construct-equipment.. Skipping
No Best log found for category: construct-pole.. Skipping
No Best log found for category: construct-post.. Skipping
No Best log found for category: construct-sign.. Skipping
best_tracked_cat_element 6.0
best_tracked_cat_log_idx 233
BEST TRACKED LOG:  daisy-2019-04-25--08-09-35 6.0
best_flickering_cat_element 11.0
best_flickering_cat_log_idx 14
BEST FLICKERING LOG:  801-frames 11.0
best_tracked_cat_element 25.0
best_tracked_cat_log_idx 45
BEST TRACKED LOG:  alanturing-2019-03-23--20-10-52--cam1 25.0
best_tracked_cat_element 3287.0
best_tracked_cat_log_idx 298
BEST TRACKED LOG:  lily-2019-04-28--09-33-

best_flickering_cat_element 1.0
best_flickering_cat_log_idx 6
BEST FLICKERING LOG:  daisy-2019-05-31--15-51-44--cam1 1.0
best_flickering_cat_element 45.0
best_flickering_cat_log_idx 96
BEST FLICKERING LOG:  bagfile_18-11-20_1935_3 45.0
best_flickering_cat_element 33.0
best_flickering_cat_log_idx 3
BEST FLICKERING LOG:  bagfile_18-10-17_1001_2 33.0
best_flickering_cat_element 150.0
best_flickering_cat_log_idx 151
BEST FLICKERING LOG:  chrysippus-2019-05-22--17-26-12--cam1 150.0
best_flickering_cat_element 4638.0
best_flickering_cat_log_idx 4
BEST FLICKERING LOG:  bagfile_18-12-08_2046_1 4638.0
best_flickering_cat_element 11740.0
best_flickering_cat_log_idx 98
BEST FLICKERING LOG:  bagfile_18-12-03_0736_13 11740.0
No Best log found for category: construct-equipment.. Skipping
No Best log found for category: construct-equipment.. Skipping
No Best log found for category: construct-pole.. Skipping
No Best log found for category: construct-pole.. Skipping
No Best log found for category: cons

No Best log found for category: traffic sign-speed_sign.. Skipping
No Best log found for category: traffic sign-stop_sign.. Skipping
No Best log found for category: traffic sign-stop_sign.. Skipping
No Best log found for category: trailer.. Skipping
No Best log found for category: trailer.. Skipping
No Best log found for category: train.. Skipping
No Best log found for category: train.. Skipping
best_flickering_cat_element 435.0
best_flickering_cat_element 3613.0
best_flickering_cat_log_idx 114
BEST FLICKERING LOG:  bagfile_18-11-18_1219_20 3613.0


Cluster: 29
best_flickering_cat_element 1.0
best_flickering_cat_log_idx 5
BEST FLICKERING LOG:  bagfile_18-11-20_0932_6 1.0
best_flickering_cat_element 36.0
best_flickering_cat_log_idx 28
BEST FLICKERING LOG:  alanturing-2019-04-12--13-06-23--cam1 36.0
best_flickering_cat_element 5.0
best_flickering_cat_log_idx 3
BEST FLICKERING LOG:  bagfile_18-10-25_1405_1 5.0
best_flickering_cat_element 530.0
best_flickering_cat_log_idx 46
BEST FLICKERIN

BEST FLICKERING LOG:  chrysippus-2019-03-20--09-04-37--cam0 433.0
best_flickering_cat_element 46.0
best_flickering_cat_log_idx 6
BEST FLICKERING LOG:  bagfile_19-01-10_0744_23 46.0
best_tracked_cat_element 358.0
best_tracked_cat_log_idx 164
BEST TRACKED LOG:  bagfile_18-11-30_0845_8 358.0
best_tracked_cat_element 1195.0
best_tracked_cat_log_idx 8
BEST TRACKED LOG:  chrysippus-2019-03-19--16-11-35--cam1 1195.0
best_flickering_cat_element 30.0
best_flickering_cat_log_idx 152
BEST FLICKERING LOG:  bagfile_18-11-16_1524_0 30.0
No Best log found for category: traffic light-amber.. Skipping
No Best log found for category: traffic light-amber.. Skipping
best_tracked_cat_element 1269.0
best_tracked_cat_log_idx 8
BEST TRACKED LOG:  daisy-2019-05-28--12-05-08--cam1 1269.0
best_tracked_cat_element 6809.0
best_tracked_cat_log_idx 172
BEST TRACKED LOG:  bagfile_18-12-13_1933_1 6809.0
best_tracked_cat_element 577.0
best_tracked_cat_log_idx 8
BEST TRACKED LOG:  lily-2019-04-12--15-55-50--cam1 577.0
b

No Best log found for category: trailer.. Skipping
No Best log found for category: train.. Skipping
No Best log found for category: train.. Skipping
best_flickering_cat_element 125.0
best_flickering_cat_log_idx 1
BEST FLICKERING LOG:  alanturing-2019-04-06--09-45-57--cam2 125.0
best_flickering_cat_element 1788.0
best_flickering_cat_log_idx 189
BEST FLICKERING LOG:  bagfile_18-09-21_0857_22 1788.0


Cluster: 20
best_flickering_cat_element 3.0
best_flickering_cat_log_idx 4
BEST FLICKERING LOG:  651-frames 3.0
best_flickering_cat_element 682.0
best_flickering_cat_log_idx 48
BEST FLICKERING LOG:  883-frames 682.0
best_flickering_cat_element 136.0
best_flickering_cat_log_idx 7
BEST FLICKERING LOG:  bagfile_18-10-25_1405_3 136.0
best_flickering_cat_element 261.0
best_flickering_cat_log_idx 81
BEST FLICKERING LOG:  alanturing-2019-02-21--14-25-14--cam1 261.0
best_flickering_cat_element 19815.0
best_flickering_cat_log_idx 7
BEST FLICKERING LOG:  bagfile_18-11-15_1051_17 19815.0
best_flickering

best_tracked_cat_element 10.0
best_flickering_cat_element 337.0
best_flickering_cat_log_idx 4
BEST FLICKERING LOG:  885-frames 337.0
No Best log found for category: bus.. Skipping
best_flickering_cat_element 39.0
best_flickering_cat_log_idx 37
BEST FLICKERING LOG:  bagfile_18-10-26_0806_3 39.0
best_flickering_cat_element 68.0
best_flickering_cat_element 5181.0
best_flickering_cat_log_idx 51
BEST FLICKERING LOG:  bagfile_18-11-27_1105_0 5181.0
No Best log found for category: construct-equipment.. Skipping
No Best log found for category: construct-equipment.. Skipping
No Best log found for category: construct-pole.. Skipping
No Best log found for category: construct-pole.. Skipping
No Best log found for category: construct-post.. Skipping
No Best log found for category: construct-post.. Skipping
No Best log found for category: construct-sign.. Skipping
No Best log found for category: construct-sign.. Skipping
No Best log found for category: motorcycle.. Skipping
best_tracked_cat_element 

No Best log found for category: traffic sign-speed_sign.. Skipping
No Best log found for category: traffic sign-stop_sign.. Skipping
No Best log found for category: traffic sign-stop_sign.. Skipping
No Best log found for category: trailer.. Skipping
No Best log found for category: trailer.. Skipping
No Best log found for category: train.. Skipping
No Best log found for category: train.. Skipping
best_flickering_cat_element 15.0
best_flickering_cat_element 1449.0
best_flickering_cat_log_idx 131
BEST FLICKERING LOG:  bagfile_19-01-08_1341_51 1449.0


Cluster: 27
best_tracked_cat_element 1.0
best_tracked_cat_log_idx 11
BEST TRACKED LOG:  bagfile_19-01-04_1217_39 1.0
best_flickering_cat_element 134.0
best_flickering_cat_log_idx 2
BEST FLICKERING LOG:  alanturing-2019-03-12--19-24-14--cam2 134.0
best_flickering_cat_element 1977.0
best_flickering_cat_log_idx 22
BEST FLICKERING LOG:  lily-2019-04-17--17-37-04--cam0 1977.0
No Best log found for category: construct-equipment.. Skipping
No Best 

In [24]:
def update_traffic_light_cats(data):
    for img_data in data:
        if img_data.get('labels', None):
            for label in [l for l in  img_data['labels'] if l['category'] == 'traffic light' and l.get('attributes', None)]:
                if label['attributes'].get('trafficLightColor', None):
                    if label['attributes']['trafficLightColor'][1].lower() == 'green' or label['attributes']['trafficLightColor'][1].lower() == 'g'  or label['attributes']['trafficLightColor'][0] == 1:
                        label['category'] = 'traffic light-green'
                    elif label['attributes']['trafficLightColor'][1].lower() == 'yellow' or label['attributes']['trafficLightColor'][1].lower() == 'y'  or label['attributes']['trafficLightColor'][0] == 2:
                        label['category'] = 'traffic light-amber'
                    elif label['attributes']['trafficLightColor'][1].lower() == 'red' or label['attributes']['trafficLightColor'][1].lower() == 'r'  or label['attributes']['trafficLightColor'][0] == 3:
                        label['category'] = 'traffic light-red'
                    else:
                        label['category'] = 'traffic light'

                # Support both old and new bdd formats
                elif label['attributes'].get('Traffic Light Color', None):
                    if label['attributes']['Traffic Light Color'][1].lower() == 'g' or label['attributes']['Traffic Light Color'][0] == 1:
                        label['category'] = 'traffic light-green'
                    elif label['attributes']['Traffic Light Color'][1].lower() == 'y' or label['attributes']['Traffic Light Color'][0] == 2:
                        label['category'] = 'traffic light-amber'
                    elif label['attributes']['Traffic Light Color'][1].lower() == 'r' or label['attributes']['Traffic Light Color'][0] == 3:
                        label['category'] = 'traffic light-red'
                    else:
                        label['category'] = 'traffic light'
    return data

In [25]:
import urllib
import ntpath
import json
from pipelineapi import PipelineAPI
import os

def get_dataset(video_source_names = [], kache_anns_dir = os.getcwd()):
    data_pipeline_api = PipelineAPI("http://ec2-54-149-236-75.us-west-2.compute.amazonaws.com/", 'admin@pronto.ai', 'admin123!')

    batch_size = 0
    batch = 0
    batch_anns = []


    dataset_names = ['032019']
    

    databunch = data_pipeline_api.data(dataset_names=dataset_names, video_source_names=video_source_names, categories_names=[],
                    label_statuses=[], page_size=None, frame_id=None) # returns generator

    for anns in databunch:
        anns = update_traffic_light_cats([anns])
        batch_anns.extend(anns)
        batch_size += len(anns)
        if batch_size % 15000 == 0:
            with open(os.path.join(kache_anns_dir,f'{"^".join(dataset_names)}_{batch}--flickers.json'), 'w') as fn:
                json.dump(batch_anns, fn)
            batch_anns = []
            batch += 1

    # Save any remaining frames
    if len(batch_anns) != 0:
        with open(os.path.join(kache_anns_dir,f'{"^".join(dataset_names)}_{batch}--flickers.json'), 'w') as fn:
            json.dump(batch_anns, fn)


In [26]:
check_s3 = True
def send_to_s3( img_path, s3_bucket = 'kache-scalabel/kache_ai/frames/'):
    s3_path = os.path.join(s3_bucket, path_leaf(img_path))
    s3_bucket = 's3://'+s3_bucket
    sp = subprocess.Popen("aws s3 cp {} {}".format(img_path, s3_bucket), shell=True, stdout=PIPE)
    out_str = sp.communicate()
    return os.path.join('https://s3-us-west-2.amazonaws.com', s3_path)

In [29]:
anns_dir = '/media/kuser/Data/071219_trn/'
os.makedirs(anns_dir, exist_ok=True)
print(set(logs_selected))   

get_dataset(video_source_names = list(set(logs_selected)), kache_anns_dir = anns_dir)

{'bagfile_19-01-13_2046_2', 'bagfile_18-11-16_1229_1', '1358-frames', 'bagfile_19-01-04_1217_39', 'alanturing-2019-04-12--14-11-23', 'daisy-2019-04-24--23-28-03--cam1', 'lily-2019-04-16--19-54-20--cam2', 'bagfile_18-11-15_1051_124', 'bagfile_18-09-21_0857_108', 'sallyride-2019-04-04--17-17-01--cam1', 'chrysippus-2019-03-27--09-10-21--cam1', 'daisy-2019-04-25--08-09-35', 'bagfile_19-01-11_0711_24', 'lily-2019-04-16--22-03-57--cam1', 'alanturing-2019-03-18--10-15-32--cam2', 'bagfile_18-12-22_0224_1', 'alanturing-2019-04-15--18-25-47--cam1', 'bagfile_18-09-21_0857_79', 'bagfile_19-01-14_1034_17', 'bagfile_18-09-21_0857_99', 'bagfile_18-09-20_1303_5', '350-frames', 'daisy-2019-04-22--18-50-43', 'lily-2019-04-12--15-55-50--cam1', 'bagfile_18-10-10_0710_4', 'alanturing-2019-03-12--19-24-14--cam2', 'chrysippus-2019-03-07--09-03-26--cam1', 'daisy-2019-04-23--00-07-17--cam1', 'bagfile_18-11-20_2029_7', 'bagfile_19-01-07_1530_24', 'daisy-2019-05-31--19-42-32--cam1', 'alanturing-2019-04-17--09-53

Traceback (most recent call last):
  File "/opt/anaconda3/envs/py36/lib/python3.6/site-packages/pipelineapi/requestsession.py", line 84, in __request
    raise Exception("502 Bad Gateway")
Exception: 502 Bad Gateway

Retry request
Traceback (most recent call last):
  File "/opt/anaconda3/envs/py36/lib/python3.6/site-packages/pipelineapi/requestsession.py", line 84, in __request
    raise Exception("502 Bad Gateway")
Exception: 502 Bad Gateway

Retry request
Traceback (most recent call last):
  File "/opt/anaconda3/envs/py36/lib/python3.6/site-packages/pipelineapi/requestsession.py", line 84, in __request
    raise Exception("502 Bad Gateway")
Exception: 502 Bad Gateway

Retry request
Traceback (most recent call last):
  File "/opt/anaconda3/envs/py36/lib/python3.6/site-packages/pipelineapi/requestsession.py", line 84, in __request
    raise Exception("502 Bad Gateway")
Exception: 502 Bad Gateway

Retry request
Traceback (most recent call last):
  File "/opt/anaconda3/envs/py36/lib/pyth

Traceback (most recent call last):
  File "/opt/anaconda3/envs/py36/lib/python3.6/site-packages/pipelineapi/requestsession.py", line 84, in __request
    raise Exception("502 Bad Gateway")
Exception: 502 Bad Gateway

Retry request
Traceback (most recent call last):
  File "/opt/anaconda3/envs/py36/lib/python3.6/site-packages/pipelineapi/requestsession.py", line 84, in __request
    raise Exception("502 Bad Gateway")
Exception: 502 Bad Gateway

Retry request
Traceback (most recent call last):
  File "/opt/anaconda3/envs/py36/lib/python3.6/site-packages/pipelineapi/requestsession.py", line 84, in __request
    raise Exception("502 Bad Gateway")
Exception: 502 Bad Gateway

Retry request
Traceback (most recent call last):
  File "/opt/anaconda3/envs/py36/lib/python3.6/site-packages/pipelineapi/requestsession.py", line 84, in __request
    raise Exception("502 Bad Gateway")
Exception: 502 Bad Gateway

Retry request
Traceback (most recent call last):
  File "/opt/anaconda3/envs/py36/lib/pyth

Traceback (most recent call last):
  File "/opt/anaconda3/envs/py36/lib/python3.6/site-packages/pipelineapi/requestsession.py", line 84, in __request
    raise Exception("502 Bad Gateway")
Exception: 502 Bad Gateway

Retry request
Traceback (most recent call last):
  File "/opt/anaconda3/envs/py36/lib/python3.6/site-packages/pipelineapi/requestsession.py", line 84, in __request
    raise Exception("502 Bad Gateway")
Exception: 502 Bad Gateway

Retry request
Traceback (most recent call last):
  File "/opt/anaconda3/envs/py36/lib/python3.6/site-packages/pipelineapi/requestsession.py", line 84, in __request
    raise Exception("502 Bad Gateway")
Exception: 502 Bad Gateway

Retry request


In [30]:
import random
import glob
import json
import numpy as np
import hashlib
import toml
import shutil
import subprocess
from subprocess import Popen,PIPE,STDOUT,call


dataset_files = glob.glob(os.path.join(anns_dir, '*.json'))
dataset_stats = {}

for idx, ann_file in enumerate(dataset_files):
    print(ann_file)
    with open(ann_file, 'r') as f:
        dataset = json.load(f)
        dataset_stats[idx] = len(dataset)
        
print(dataset_stats)
tot_frames = np.sum(list(dataset_stats.values()))
print(tot_frames)

remove_count = tot_frames - 30000 # labeling budget (30k in production)

if remove_count > 0:
    databatch_portions = [float(datalen/tot_frames) for datalen in dataset_stats.values()]
    remove_counts = [int(remove_count*portion)+1 for portion in databatch_portions]
    
    # Randomly remove frames based on remove count
    for idx, ann_file in enumerate(dataset_files):
        with open(ann_file, 'r') as f:
            dataset = json.load(f)
        frames = list(range(len(dataset)))
        random.shuffle(frames)
        deleted_frames = frames[:remove_counts[idx]]
        dataset = [img_data for idx, img_data in enumerate(dataset) if  idx not in deleted_frames]
        with open(ann_file, 'w+') as f:
            json.dump(dataset, f)
            

# Change name/url to s3_url
for idx, ann_file in enumerate(dataset_files):
    
    with open(ann_file, 'r') as f:
        dataset = json.load(f)

    for img_data in dataset:
        hex_val = os.path.abspath(img_data['url']).replace('/', '^^')
        hash_object = hashlib.sha1(str.encode(hex_val))
        hex_dig = hash_object.hexdigest()

        tmp_img_dir = os.path.join(anns_dir, 'images')
        os.makedirs(tmp_img_dir, exist_ok = True)
        orig_frame_path = img_data['url']
        with open('/media/kuser/Data/nebula/pipelines/configs/entanglement_cfg.tml', 'r') as tmlfile:
            cfg_data = toml.load(tmlfile)

        orig_frame_path = os.path.join(cfg_data['entanglement']['master_nebula_dir'], img_data['url'].split('nebula')[-1].strip('/') )
        new_path = os.path.join(tmp_img_dir, str(hex_dig)+os.path.splitext(path_leaf(img_data['url']))[-1])

        shutil.copyfile(orig_frame_path, new_path)
        s3uri = send_to_s3( new_path )

        img_data['url'] = s3uri



    with open(ann_file, 'w+') as f:
        json.dump(dataset, f)
    
    

/media/kuser/Data/071219_trn/032019_0--flickers.json
/media/kuser/Data/071219_trn/032019_1--flickers.json
/media/kuser/Data/071219_trn/032019_10--flickers.json
/media/kuser/Data/071219_trn/032019_11--flickers.json
/media/kuser/Data/071219_trn/032019_12--flickers.json
/media/kuser/Data/071219_trn/032019_13--flickers.json
/media/kuser/Data/071219_trn/032019_14--flickers.json
/media/kuser/Data/071219_trn/032019_15--flickers.json
/media/kuser/Data/071219_trn/032019_16--flickers.json
/media/kuser/Data/071219_trn/032019_2--flickers.json
/media/kuser/Data/071219_trn/032019_3--flickers.json
/media/kuser/Data/071219_trn/032019_4--flickers.json
/media/kuser/Data/071219_trn/032019_5--flickers.json
/media/kuser/Data/071219_trn/032019_6--flickers.json
/media/kuser/Data/071219_trn/032019_7--flickers.json
/media/kuser/Data/071219_trn/032019_8--flickers.json
/media/kuser/Data/071219_trn/032019_9--flickers.json
{0: 15000, 1: 15000, 2: 15000, 3: 15000, 4: 15000, 5: 15000, 6: 15000, 7: 15000, 8: 14520, 9